## ${\color{Blue} \text{ Part I: Spectrum Estimation }}$

**Review**

Given N samples ${\{y_0, y_1, . . . , y_{N−1}\}}$, the periodogram is given by

$$
\begin{equation}
\hat{S}_{yy}(\mathit{f}) = \hat{S}_{PER}(\mathit{f}) = \frac{1}{N} \left| \sum_{n=0}^{N-1} w_{N,n} y_n e^{-j2\pi fn}  \right|^2
\end{equation}
$$

where $w_{N,n}$ is a windowing function of size $N$ (rectangular by default). The window is normalized in the sense that $\frac{1}{N} \sum_{n=0}^{N-1} w_{N,n}^2 = 1$. In practice, the Fourier Transform (FFT)is computed as the Discrete Fourier Transform (DFT), for which fast algorithms exist known as Fast Fourier Transform (FFT). For computation of the DFT/FFT, it is customary to consider the frequency interval $[0, 1]$ rather than $[−\frac{1}{2}, \frac{1}{2} ]$. The DFT evaluates the Fourier transform of a signal of length $N$ at $N$ equispaced frequencies $f_k = k/N , k = 0,1,...,N−1$. So we get
$$
\begin{equation}
\hat{S}_{yy}(\mathit{f}) = \hat{S}_{PER}(\mathit{f}) = \frac{1}{N} \left| \sum_{n=0}^{N-1} w_{N,n} y_n e^{-j2\pi fn}  \right|^2 = \frac{1}{N} \left| \sum_{n=0}^{N-1} w_{N,n} y_n e^{-j2\pi \frac{k}{N}n}  \right|^2, k = 0,1,...,N−1 .
\end{equation}
$$
We can obtain a finer frequency spacing by padding the data with $N' - N$ zeros and then applying an $N′$-point DFT.  The effective data set becomes
$$
\begin{equation}
y'_n = 
\begin{cases}
&y_n &, n = 0,1, ..., N - 1 \\
&0 &, n = N, N+1, ..., N' - 1
\end{cases}
\end{equation}
$$
which has the same Fourier transform as the original data set. The frequency spacing of the DFT on the data set $y′$ will be $\frac{1}{N'} < \frac{1}{N}$ . This **zero** padding gives no extra resolution, but only a finer evaluation of the periodogram. The window is still of size $N$.

In the autoregressive (AR) modeling (parametric) approach, we take
$$
\begin{equation}
\hat{S}_{AR}(\mathit{f}) =  \frac{\sigma^2_{f,n}}{|A_n(f)|^2}
\end{equation}
$$
for an AR model of order n. The prediction coefficients $A_n = [1 A_{n,1} · · · A_{n,n}]T$ and the prediction error variance are obtained from the Yule-Walker (or normal) equations
$$
R_{n+1}A_n = 
 \begin{bmatrix}
  r_0 & r_1 & \cdots & r_n \\
  r_1 & \cdots & \cdots & \vdots \\
  \vdots  & \cdots  & \ddots & r_1  \\
  r_n & \cdots & r_1 & r_0
 \end{bmatrix} = 
 \begin{bmatrix}
  1 \\
  A_{n,1} \\
  \vdots \\
  A_{n,n} 
 \end{bmatrix} =
 \begin{bmatrix}
  \sigma_{f,n}^2 \\
  0 \\
  \vdots \\
  0
 \end{bmatrix}
$$
where the correlation sequence is estimated as.
$$
r_k = \hat{r}_{yy}(k) = \frac{1}{N} \sum_{n=0}^{N-1-k} y_n+ky_n, k = 0, 1, \cdots, n .
$$

The frequencies f in (4) are again evaluated at the same discrete set. The prediction filter can be computed in an order-recursive $\Delta_{n+1}$ fashion via the $\underline{Levinson \; algorithm}$: 
$
\quad
\begin{cases} 
A_n \\
\sigma_{f,n}^2
\end{cases}
$
$
\implies
\begin{cases} 
A_{n+1} \\
\sigma_{f,n+1}^2
\end{cases}
$


$$
\begin{gather}
\Delta_{n+1}= [r_{n+1} \cdots r_1] A_n
\\
K_{n+1} = -\frac{\Delta_{n+1}}{\sigma_{f,n}^2} \qquad
\\
\qquad \qquad
A_{n+1} = 
\begin{bmatrix}
A_n \\
0
\end{bmatrix} +
K_{n+1}
\begin{bmatrix}
0 \\
J A_n
\end{bmatrix}
\\
\sigma_{f,n+1}^2 = \sigma_{f,n}^2 \left( 1 - K_{n+1}^2 \right)
\end{gather}
$$

Initialisation $A_0 = [1], \sigma_{f,0}^2 = r_0$

## ${\color{Blue} \text{ Part II: Parameter Estimation }}$

### Background on ML Estimation of Sinusoid in Noise Parameters

For this second part, consider a single sinusoid in white Gaussian noise with variance $\sigma^2$
$$
y_k =s_k +v_k = A_1 \cos(2\pi f_1 k + \phi_1) + v_k \; , \; k = 0, \cdots , n − 1 .
$$
The Maximum Likelihood estimates of the paramteres $\sigma^2, A_1, \phi_1$ and $f_1$ can be shown to be obtained as
$$
\begin{split}
\hat{f}_1 = \operatorname*{arg\,max}_f |\mathcal{Y}(f)| \qquad \qquad \qquad \qquad
\\
\hat{A}_1 = \frac{2}{n} | \mathcal{Y}(\hat{f}_1) | \qquad \qquad \qquad \qquad \qquad 
\\
\hat{\phi}_1 = \operatorname*{arg}_{n - 1} \mathcal{Y}(\hat{f}_1) \qquad \qquad \qquad \qquad \qquad 
\\
\widehat{\sigma^2} = \frac{1}{n} \sum_{k=0}^{n-1} (y_k − \hat{A}_1 \cos(2\pi \hat{f}_1 k + \hat{\phi}_1))^2
\end{split}
$$
where $Y(f)= \sum_{k=0}^{n-1} y_k e^{−j 2 \pi f k} $ and $ \operatorname{arg} \{ \rho e^{j \theta} \} = \theta$. In order to turn the optimization problem $k=0$ for $\hat{f}_1$ into a practical algorithm, we shall use the DFT. First decide on an acceptable “bias” in the ability to resolve the maximum of $|\mathcal{Y}(f)|$. Let’s call the frequency resolution $\Delta f$ :
$$ \Delta f = \frac{1}{m}, \; m = \frac{1}{\Delta f} .$$

In order to have a DFT with such a frequency resolution, we need to have a signal of length $m$. We assume $m > n$, the number of samples available. Hence zero pad y_k to obtain $y_0,y_1,...,y_{m−1}$ where in fact $y_n = y_{n+1} = y_{m−1} = 0$. Take the DFT of the zero padded sequence (in Matlab, y_0 is the first element of a signal vector, y_1 the second etc.)
$$Y_l = \sum_{k=0}^{m-1} y_k e^{j 2 \pi \frac{l}{m} k} = \sum_{k=0}^{n-1} y_k e^{j 2 \pi \frac{l}{n} k} .$$
Then the Maximum Likelihood estimates can be approximately obtained as
$$
\begin{split}
\hat{l} = \operatorname*{arg\,max}_l |\mathcal{Y}_l|, \qquad \hat{f}_1 = \frac{\hat{l}}{m}, \qquad \hat{A}_1 = \frac{2}{n} | \mathcal{Y}_\hat{l} | \qquad \qquad 
\\
\hat{\phi}_1 = \operatorname*{arg} \mathcal{Y}\hat{l}, \quad 
\widehat{\sigma^2} = \frac{1}{n} \sum_{k=0}^{n-1} (y_k − \hat{A}_1 \cos(2\pi \hat{f}_1 k + \hat{\phi}_1))^2
\end{split}
$$


The search for $f_1$ should be limited to the interval $[0, \frac{1}{2} ]$ (the DFT will show a symmetrical peak at $1-\hat{f}_1$ ). Note that zeropadding only allows us to get within $\Delta f = \frac{1}{m}$ of the maximum of $|\mathcal{Y}(f)|$. It does not improve the estimation accuracy (variance) of the estimator $\hat{f}_1$. The variance can only be reduced by increasing the number of samples $n$ (see Cramer-Rao bound).

The Cramer-Rao bounds for the estimation of the various parameters can be shown to be:

$$
\begin{gather}
CRB_{\widehat{\sigma^2}} = \frac{2 \sigma^4}{n} \qquad CRB_{\hat{A}_1} = \frac{2 \sigma^2}{n} \\
CRB_{\widehat{\phi_1}} = \frac{8 \sigma^2}{n A_1^2 } \qquad CRB_{\widehat{f_1}} = \frac{6 \sigma^2}{\pi^2 n^3 A_1^2 }
\end{gather}
$$